In [1]:
from scipy.misc import imread
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dense, Activation, Flatten, Input, Dropout
from keras.applications.vgg16 import decode_predictions
from keras import optimizers

from scipy.misc import imresize
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


In [28]:
train_path = '/home/ubuntu/project5/trees/train/'
valid_path = '/home/ubuntu/project5/trees/test/'
test_path = '/home/ubuntu/project5/trees/valid/'

In [4]:
#train_path = '/home/ubuntu/project5/trees/train/'
#valid_path = '/home/ubuntu/project5/trees/test/'
#test_path = '/home/ubuntu/project5/trees/valid/'
image_size = 224

vgg_model = VGG16(weights='imagenet', include_top=False, input_shape = (image_size, image_size, 3))

for layer in vgg_model.layers[:-4]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in vgg_model.layers:
    print(layer, layer.trainable)


model = Sequential()

# Add the vgg convolutional base model
model.add(vgg_model)

# Add new layers
model.add(Flatten())
model.add(Dropout(0.6))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

58892288/58889256 [==============================] - 57s 1us/step
<keras.engine.topology.InputLayer object at 0x1c137c39e8> False
<keras.layers.convolutional.Conv2D object at 0x180bb85278> False
<keras.layers.convolutional.Conv2D object at 0x1c137c3d30> False
<keras.layers.pooling.MaxPooling2D object at 0x1c15a4f7f0> False
<keras.layers.convolutional.Conv2D object at 0x1c15ad0780> False
<keras.layers.convolutional.Conv2D object at 0x1c15b1e278> False
<keras.layers.pooling.MaxPooling2D object at 0x1c15c41a20> False
<keras.layers.convolutional.Conv2D object at 0x1c15c41eb8> False
<keras.layers.convolutional.Conv2D object at 0x1c15b2dcc0> False
<keras.layers.convolutional.Conv2D object at 0x1c15b2dd68> False
<keras.layers.pooling.MaxPooling2D object at 0x1c15852240> False
<keras.layers.convolutional.Conv2D object at 0x1c16215e48> False
<keras.layers.convolutional.Conv2D object at 0x180bb96828> False
<keras.layers.convolutional.Conv2D object at 0x1c16541550> False
<keras.layers.pooling.Max

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              51382272  
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
__________

In [30]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=90,
        width_shift_range=0.0,
        height_shift_range=0.0,
        shear_range=0.0,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_batchsize = 50
valid_batchsize = 10
test_batchsize = 10

train_gen = train_datagen.flow_from_directory(train_path, 
                            target_size = (image_size, image_size), 
                            batch_size = train_batchsize,
                            class_mode = 'categorical')

valid_gen = valid_datagen.flow_from_directory(valid_path,
                            target_size = (image_size, image_size),
                            batch_size = valid_batchsize,
                            class_mode = 'categorical',
                            shuffle = False)

test_gen = test_datagen.flow_from_directory(test_path,
                            target_size = (image_size, image_size),
                            batch_size = test_batchsize,
                            class_mode = 'categorical',
                            shuffle = False)


FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/project5/trees/train/'

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-6),
              metrics=['acc'])

history = model.fit_generator(
      train_gen,
      steps_per_epoch=2*train_gen.samples/train_gen.batch_size ,
      epochs=20,
      validation_data=valid_gen,
      validation_steps=valid_gen.samples/valid_gen.batch_size,
      verbose=1)

model.save('model_vgg16_test8.model')
#model.save_weights('tree_model_1_weights.wts')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
epochs = range(len(acc))

In [ ]:
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

In [ ]:
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

In [ ]:
filename = valid_gen.filenames
truth = valid_gen.classes
label = valid_gen.class_indices
indexlabel = dict((value, key) for key, value in label.items())

In [ ]:
predicts = model.predict_generator(valid_gen, steps=valid_gen.samples/valid_gen.batch_size, verbose=1)
predict_class = np.argmax(predicts, axis=1)
errors = np.where(predict_class != truth)[0]
print("No of errors = {}/{}".format(len(errors),valid_gen.samples))

In [ ]:
for i in range(len(errors)):
    pred_class = np.argmax(predicts[errors[i]])
    pred_label = indexlabel[pred_class]
    
    title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        filename[errors[i]].split('/')[0],
        pred_label,
        predicts[errors[i]][pred_class])
    
    original = load_img('{}/{}'.format(valid_path,filename[errors[i]]))
    print(filename[errors[i]])
    plt.figure(figsize=[7,7])
    plt.axis('off')
    plt.title(title)
    plt.imshow(original)
    plt.show()

In [ ]:
test_file = test_gen.filenames
test_truth = test_gen.classes
test_label = test_gen.class_indices
test_indexlabel = dict((value,key) for key, value in test_label.items())

In [ ]:
test_pred = model.predict_generator(test_gen, steps=test_gen.samples/test_gen.batch_size, verbose=1)
test_pred_class = np.argmax(test_pred, axis=1)
test_error = np.where(test_pred_class != test_truth)[0]
print('Number of errors = {}/{}'.format(len(test_error),test_gen.samples))

In [ ]:
test_error

In [ ]:
for i in range(len(test_error)):
    test_predclass = np.argmax(test_pred[test_error[i]])
    test_predlabel = test_indexlabel[test_predclass]
    
    title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        test_file[test_error[i]].split('/')[0],
        test_predlabel,
        test_pred[test_error[i]][test_predclass])
    
    test_original = load_img('{}/{}'.format(test_path,test_file[test_error[i]]))
    print(test_file[test_error[i]])
    plt.figure(figsize=[7,7])
    plt.axis('off')
    plt.title(title)
    plt.imshow(test_original)
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(truth,predict_class)

labels = []
for k,v in indexlabel.items():
    labels.append(v)
    
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

plt.figure(figsize=(10,10))
plot_confusion_matrix(cm, classes=labels,
                      title='Confusion matrix, without normalization')

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_truth,test_pred_class)

labels = []
for k,v in test_indexlabel.items():
    labels.append(v)
    
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

plt.figure(figsize=(10,10))
plot_confusion_matrix(cm, classes=labels,
                      title='Confusion matrix, without normalization')

In [27]:
model = keras.models.load_model('model_vgg16_11.model')

In [21]:
test_path = ('./picts/')
image_size = 244
batch_size = 10

In [25]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_gen = test_datagen.flow_from_directory(test_path,
                            target_size = (image_size, image_size),
                            batch_size = batch_size,
                            class_mode = 'categorical',
                            shuffle = False)


Found 1023 images belonging to 7 classes.


In [26]:
test_pred = model.predict_generator(test_gen, steps=test_gen.samples/test_gen.batch_size, verbose=1)
test_pred_class = np.argmax(test_pred, axis=1)
test_error = np.where(test_pred_class != test_truth)[0]
print('Number of errors = {}/{}'.format(len(test_error),test_gen.samples))

ValueError: Error when checking : expected vgg16_input to have shape (None, 224, 224, 3) but got array with shape (10, 244, 244, 3)